In [33]:
# ✅ STEP 1: Install Required Libraries
!pip install pandas numpy bokeh panel pathway --quiet

# ✅ STEP 2: Import Required Modules
import pandas as pd
import numpy as np
import panel as pn
import bokeh.plotting
from bokeh.models import ColumnDataSource
pn.extension()

# ✅ STEP 3: Load the Dataset
df = pd.read_csv("dataset.csv")

# ✅ STEP 4: Combine Date & Time columns into Timestamp
df['Timestamp'] = pd.to_datetime(df['LastUpdatedDate'] + ' ' + df['LastUpdatedTime'],
                                  format='%d-%m-%Y %H:%M:%S')

# ✅ STEP 5: Display the First Few Rows
df.head()


,ID,SystemCodeNumber,Capacity,Latitude,Longitude,Occupancy,VehicleType,TrafficConditionNearby,QueueLength,IsSpecialDay,LastUpdatedDate,LastUpdatedTime,Timestamp
0,0,BHMBCCMKT01,577,26.144536,91.736172,61,car,low,1,0,04-10-2016,07:59:00,2016-10-04 07:59:00
1,1,BHMBCCMKT01,577,26.144536,91.736172,64,car,low,1,0,04-10-2016,08:25:00,2016-10-04 08:25:00
2,2,BHMBCCMKT01,577,26.144536,91.736172,80,car,low,2,0,04-10-2016,08:59:00,2016-10-04 08:59:00
3,3,BHMBCCMKT01,577,26.144536,91.736172,107,car,low,2,0,04-10-2016,09:32:00,2016-10-04 09:32:00
4,4,BHMBCCMKT01,577,26.144536,91.736172,150,bike,low,2,0,04-10-2016,09:59:00,2016-10-04 09:59:00


In [34]:
# ✅ MODEL 1: BASELINE LINEAR MODEL
# Base price for all lots
BASE_PRICE = 10

# Occupancy rate = Occupied / Capacity
df['OccupancyRate'] = df['Occupancy'] / df['Capacity']
df['Price_Model1'] = BASE_PRICE * (1 + 0.5 * df['OccupancyRate'])

# ✅ Show Result
df[['SystemCodeNumber', 'Timestamp', 'Occupancy', 'Capacity', 'OccupancyRate', 'Price_Model1']].head()


,SystemCodeNumber,Timestamp,Occupancy,Capacity,OccupancyRate,Price_Model1
0,BHMBCCMKT01,2016-10-04 07:59:00,61,577,0.105719,10.528596
1,BHMBCCMKT01,2016-10-04 08:25:00,64,577,0.110919,10.554593
2,BHMBCCMKT01,2016-10-04 08:59:00,80,577,0.138648,10.693241
3,BHMBCCMKT01,2016-10-04 09:32:00,107,577,0.185442,10.927210
4,BHMBCCMKT01,2016-10-04 09:59:00,150,577,0.259965,11.299827


In [35]:
# ✅ MODEL 2: DEMAND-BASED PRICE FUNCTION

# Traffic condition mapping
traffic_map = {'low': 0, 'medium': 1, 'high': 2}
vehicle_map = {'bike': 0.5, 'car': 1.0, 'truck': 1.5}

# Map traffic and vehicle type
df['TrafficLevel'] = df['TrafficConditionNearby'].map(traffic_map)
df['VehicleWeight'] = df['VehicleType'].map(vehicle_map)

# Calculate demand score
df['DemandScore'] = (
    0.4 * df['OccupancyRate'] +
    0.2 * df['QueueLength'] / 10 -
    0.1 * df['TrafficLevel'] +
    0.15 * df['IsSpecialDay'] +
    0.15 * df['VehicleWeight']
).clip(0, 1)

# Calculate price
df['Price_Model2'] = 10 * (1 + 0.5 * df['DemandScore'])

# ✅ Show Result
df[['SystemCodeNumber', 'Timestamp', 'Price_Model2']].head()


In [37]:
# ✅ MODEL 3: COMPETITIVE PRICING MODEL

# Round timestamp to 5-minute buckets to simulate batches
df['TimeBucket'] = df['Timestamp'].dt.floor('5min')

# Compute average price of Model 2 in the same time bucket
avg_price_per_time = df.groupby('TimeBucket')['Price_Model2'].transform('mean')

# Apply boost if demand is high
df['Price_Model3'] = avg_price_per_time + np.where(df['DemandScore'] > 0.7, 1.0, -0.5)

# ✅ Show Result
df[['SystemCodeNumber', 'Timestamp', 'Price_Model3']].head()


,SystemCodeNumber,Timestamp,Price_Model3
0,BHMBCCMKT01,2016-10-04 07:59:00,10.891819
1,BHMBCCMKT01,2016-10-04 08:25:00,11.089649
2,BHMBCCMKT01,2016-10-04 08:59:00,11.274982
3,BHMBCCMKT01,2016-10-04 09:32:00,11.379678
4,BHMBCCMKT01,2016-10-04 09:59:00,11.677519


In [39]:
from bokeh.plotting import figure, show, output_notebook
from bokeh.models import ColumnDataSource, HoverTool
from bokeh.palettes import Category10

output_notebook()

# ✅ Select a sample lot
selected_lot = 'BHMBCCMKT01'
lot_df = df[df['SystemCodeNumber'] == selected_lot].copy()

# ✅ Prepare Bokeh data source
source = ColumnDataSource(data=dict(
    time=lot_df['Timestamp'],
    price1=lot_df['Price_Model1'],
    price2=lot_df['Price_Model2'],
    price3=lot_df['Price_Model3'],
))

# ✅ Create Bokeh figure
p = figure(x_axis_type='datetime', width=850, height=400, title=f"Daily Price Comparison – Lot {selected_lot}")
p.line('time', 'price1', source=source, color=Category10[3][0], legend_label="Model 1 – Baseline", line_width=2)
p.line('time', 'price2', source=source, color=Category10[3][1], legend_label="Model 2 – Demand-Based", line_width=2)
p.line('time', 'price3', source=source, color=Category10[3][2], legend_label="Model 3 – Competitive", line_width=2)

p.add_tools(HoverTool(tooltips=[
    ("Time", "@time{%F %T}"),
    ("Model 1", "@price1"),
    ("Model 2", "@price2"),
    ("Model 3", "@price3")
], formatters={'@time': 'datetime'}))

p.legend.location = 'top_left'
p.xaxis.axis_label = 'Timestamp'
p.yaxis.axis_label = 'Predicted Price ($)'

# ✅ Show the figure
show(p)


PATHWAY PROGRESS DASHBOARD                                             
                                                                                                                   
                         no.                                           latency                                     
                    messages                                           to wall     lag to                          
                      in the                                             clock      input      total    current    
                        last   in the last        since    operator       [ms]       [ms]       rows       rows    
    connector      minibatch        minute        start   ──────────────────────────────────────────────────────   
   ─────────────────────────────────────────────────────   input           476                                     
    FileSystem…            0             0        18368    output          476          0                          
    FileSystem…            0             0        18368                                                            
    FileSystem…            0             0        18368     Above you can see the latency of input and output      
                                                           operators. The latency is measured as the difference    
                                                          between the time when the operator processed the data    
                                                               and the time when pathway acquired the data.        
                                                                                                                   
                                                                                                                   
                                                                                                                   
                                                                                                                   
                                                                                                                   
                                                                                                                   
                                                                                                                   
                                                                                                                   
                                                                                                                   
                                                                                                                   
                                                                                                                   
                                                                                                                   
                                                                                                                   
                                                                                                                   
                                                                                                                   
                                                                                                                   
                                                                                                                   
                                                                                                                   
                                                                                                                   
                                                                                                                   
                                                                                                    

In [40]:
!pip install pathway panel bokeh --quiet


In [41]:
# Install required libraries (only needed once)
!pip install -q pathway panel bokeh

# Imports
import pathway as pw
import panel as pn
import bokeh.plotting
pn.extension()

# Define the schema
class ParkingSchema(pw.Schema):
    SystemCodeNumber: str
    Capacity: int
    Occupancy: int
    QueueLength: int
    TrafficConditionNearby: str
    VehicleType: str
    IsSpecialDay: int
    Latitude: float
    Longitude: float
    LastUpdatedDate: str
    LastUpdatedTime: str

# Define pricing model (Model 2: Demand-Based Function)
@pw.udf
def compute_dynamic_price(cap, occ, qlen, traffic, special, vtype):
    cap = cap if cap else 1
    occ_rate = occ / cap
    traffic_map = {'low': 0, 'medium': 1, 'high': 2}
    vehicle_map = {'bike': 0.5, 'car': 1.0, 'truck': 1.5}
    t = traffic_map.get(traffic, 1)
    v = vehicle_map.get(vtype, 1.0)
    demand = 0.4 * occ_rate + 0.2 * qlen / 10 - 0.1 * t + 0.15 * special + 0.15 * v
    demand = min(1.0, max(0.0, demand))
    return round(10 * (1 + 0.5 * demand), 2)

# Stream from dataset.csv using Pathway
stream = pw.io.csv.read(
    "dataset.csv",
    schema=ParkingSchema,
    mode="streaming",
    autocommit_duration_ms=500,
)

# Apply pricing logic
delta_window = stream.select(
    t=pw.this.LastUpdatedDate + " " + pw.this.LastUpdatedTime,
    price=compute_dynamic_price(
        pw.this.Capacity,
        pw.this.Occupancy,
        pw.this.QueueLength,
        pw.this.TrafficConditionNearby,
        pw.this.IsSpecialDay,
        pw.this.VehicleType
    )
)

# Real-time Bokeh plot function
def price_plotter(source):
    fig = bokeh.plotting.figure(
        height=400,
        width=800,
        title="Pathway: Real-Time Parking Price",
        x_axis_type="datetime",
    )
    fig.line("t", "price", source=source, line_width=2, color="navy")
    fig.circle("t", "price", source=source, size=6, color="red")
    return fig

# Create and display real-time plot with Panel
viz = delta_window.plot(price_plotter, sorting_col="t")
pn.Column(viz).servable()


Column
    [0] Column
        [0] Row
            [0] Markdown(str)
            [1] TooltipIcon(value='Table depends o...)
        [1] Bokeh(figure)

PATHWAY PROGRESS DASHBOARD                                             
                                                                                                                   
                         no.                                           latency                                     
                    messages                                           to wall     lag to                          
                      in the                                             clock      input      total    current    
                        last   in the last        since    operator       [ms]       [ms]       rows       rows    
    connector      minibatch        minute        start   ──────────────────────────────────────────────────────   
   ─────────────────────────────────────────────────────   input           429                                     
    FileSystem…            0             0        18368    output          429          0                          
    FileSystem…            0             0        18368                                                            
    FileSystem…            0             0        18368     Above you can see the latency of input and output      
                                                           operators. The latency is measured as the difference    
                                                          between the time when the operator processed the data    
                                                               and the time when pathway acquired the data.        
                                                                                                                   
                                                                                                                   
                                                                                                                   
                                                                                                                   
                                                                                                                   
                                                                                                                   
                                                                                                                   
                                                                                                                   
                                                                                                                   
                                                                                                                   
                                                                                                                   
                                                                                                                   
                                                                                                                   
                                                                                                                   
                                                                                                                   
                                                                                                                   
                                                                                                                   
                                                                                                                   
                                                                                                                   
                                                                                                                   
                                                                                                    

In [42]:
%%capture --no-display
pw.run()


LiveError: Only one live display may be active at once